In [45]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import os
import sys
import json
from psycopg2.extras import Json
#import psycopg2
#import psycopg2.extras


#Importação das funçoes da pasta "modules" (aqui, provisioriamente copiada para a pasta do BinanceTrader)
from modules import db_functions as dbf

In [15]:
#Credenciais do banco

POSTGRES_DB='pg_dev'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'
POSTGRES_USER='openbot'
DB_HOST='localhost'
DB_PORT='5432'

db_url = 'postgresql://' + POSTGRES_USER + ':' + POSTGRES_PASSWORD + '@' + DB_HOST + ':' + DB_PORT + '/' + POSTGRES_DB

engine = create_engine(db_url,echo=False)

In [52]:
# Usar para criar a tabela da estratégia com seus parâmetros, a patir do arquivo json, caso não exista


#Importar json config

config_path = 'data_structure/alpha_setup_1.json'

data_in = open(config_path)

config = json.loads(data_in.read()) #Dicionário com os parâmetros de configuração

data_in.close()


#Tabela da estratégia (Provisioriamente. O django é quem deve gerenciá-la)


#Colunas
keys = {'name': 'text', 'work_pairs': 'text []', 'candle_interval': 'text', 'price_source': 'text', 'indicators_set': 'json', 'stop_criteria': 'json', 'update_target_if': 'json'}

#Dados para salvar
dic_in = dict(config['setup_1'])

data_to_save = []

data_to_save.append('setup_1')

data_to_save.append(dic_in['work_pairs'])

data_to_save.append(str(dic_in['candle_interval']))

data_to_save.append(str(dic_in['price_source']))

data_to_save.append(json.dumps(dic_in['indicators_set']))

data_to_save.append(json.dumps(dic_in['stop_criteria']))

data_to_save.append(json.dumps(dic_in['update_target_if']))


#Criando a tabela
table_name = 'strategy_alpha'

create_table_job_satus = dbf.create_table(table_name,keys,pk='name')

data_to_save = [data_to_save]


#Salvar estratégia na tabela
save_in_table_job_status = dbf.save_in_table(table_name, keys, data_to_save)

## Funções auxiliares

### Parseamento de candles

In [5]:
def candle_parse():
    pass

In [61]:
#Teste parseamento candles

## Indicators

In [6]:
# Apenas o cálculo da média móvel, dada uma massa de dados e um "n_sample"

def simple_ma (data, n_samples):
    
    '''Simple moving average. From a pandas data series, returns the last value of rolling mean of the 
    last <n_samples> samples.
    
    '''
    
    rolling_mean = data.rolling(window=int(n_samples)).mean()
    
    last_sample_sma = rolling_mean[len(rolling_mean)-1]
    
    return last_sample_sma

In [46]:
#Teste indicadores

## Strategies

In [7]:
#Apenas estratégia alpha (cruzamento das médias móveis) por hora

from modules import indicators

def alpha(data, n_short, n_long):
    
    relative_size = 1
    
    short_sma = simple_ma(data, n_short)
    
    long_sma = simple_ma(data, n_long)
    
    if (short_sma > long_sma): 
        
        side = 'asset'
    
    else:
        
        side = 'quote'
    
    return short_sma, long_sma, side, relative_size

In [13]:
#Teste estratégias

pair = 'BTCUSDT'

data = data_dict[pair]['close']

alpha(data, 3, 100)

#print(result)

(10846.920000000018, 10770.611800000004, 'asset', 1)

# BinanceTrader

## Algoritmo

### Entrada

#### Backtest

> 1 - Recebe a definição da estratégia e do setup, carregando os parâmetros do setup da tabela da estratégia **(esta tabela DEVE existir)**  
> 2 - Busca a tabela de log operacional  
    >> A - Se não houver, a cria  
    >> B - Se existir, carregar dados da última linha  

> 3 - Lê as tabelas de dados:  
    >> A - Inteiras, caso acima seja A  
    >> B - Parciais, **suficientemente longas (obs: Será preciso uma função para determinar este comprimento mínimo, a partir dos parâmetros dos indicadores, bem como do candle_interval utilizados) tomando como referência à última amostra trazida, caso acima seja B**  

> 4 - Cria o dicionário {'par': 'market_dataframe'} **(deve "normalizar os dados para que todos os pares iniciem do mesmo 'open_time', caso a cadeia *A* acima se realize)**  
> 5 - Itera a estratégia, correndo do primeiro open_time ao último, para todos os pares do dicionário:  
    >> 5.1 - Confere os critérios de stop para os pares comprados  
    >> 5.2 - Aplica a estratégia propriamente dita, a fim de verificar a posição esperada  
    >> 5.3 - Compara a posição esperada com a posição corrente  
    >> 5.4 - Realiza compras ou vendas, em caso de divergência das posições (esperada e corrente)
    

### Real time test ou trade

> 1 - Recebe a definição da estratégia e localiza seus parâmetros na tabela apropriada  
> 2 - Lê as tabelas de dados Parciais e suficientemente longas para o caso de trade real **(obs: Será preciso uma função para determinar este comprimento mínimo, a partir dos parâmetros dos indicadores, bem como do candle_interval utilizados)**

## Levantamento de parâmetros

In [23]:
#PROVISÓRIOS
#Migrar, assim que possível para banco de dados

#candle_interval = '1m'
strategy = 'alpha'
setup = "setup_1"

In [71]:
#Buscando parâmetros

table_name = 'strategy_' + strategy

sql_query = "SELECT * FROM " + table_name + " WHERE name = '" + setup + "';"

setups = pd.read_sql(sql_query, engine)

setup_parameters = setups.loc[setups.name == setup]

#candle_interval = setup_parameters['candle_interval'][0]
#price_source = 
#setup_parameters['stop_criteria'][0]['first_trigger']['percent']

In [74]:
type(setup_parameters['work_pairs'][0])

list

## Leitura Dados de Mercado

In [11]:
#Leitura dos dados de mercado
    # - Tabelas inteiras para backetest ou
    # - Parte das tabelas, suficientemente longa para cálculo dos indicadores (ciclos)

    
def get_klines_from_db(pairs_list, from_date_time):
    pass

table_name = 'binance_pairs'

sql_query = 'SELECT * FROM ' + table_name

pairs = pd.read_sql(sql_query, engine)

data_dict = {}

for i in range(len(pairs)):
        
    if ((pairs.iloc[i]['get_data'] == 'ON') and (pairs.iloc[i]['status'] == 'full')):
        
        #(por hora,os candles serão "artificialmente gerados a partir do candle de 1m")
        table_name = 'binance_klines_' + str(pairs.iloc[i]['name']) + '_1m' #+ candle_interval 
        
        sql_query = 'SELECT * FROM ' + table_name

        #Dicionário {'par': 'candles_dataframe'}
        data_dict[ pairs.iloc[i]['name'] ] = pd.read_sql(sql_query, engine)

In [76]:
#Teste: Localizando um dado pelo open_time

dados = data_dict['BTCUSDT']

open_time = '2018-07-02 20:05:00'

dados.loc[dados.open_time == open_time]

#float(dados['close'].loc[dados.open_time == open_time])

,open_time,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
457004,2018-07-02 20:05:00,6624.9,6626.0,6622.09,6625.79,18.110122,119966.445511,159,14.081263,93281.834906


In [80]:
data_dict.keys()

dict_keys(['ETHUSDT', 'XRPUSDT', 'LINKUSDT', 'NEOUSDT', 'BTCUSDT', 'LTCUSDT'])

## Main

In [ ]:
'''
A cada ciclo (iniciando de uma posição sucifientemente longa para poder calcular o valor do indicador) avalia 
em que posição deve estar o ativo, etc (COMPLETAR DOCSTRING)
  
Caso não exista o log operacional, tanto a tabela, quanto a primeira linha (todos os pares na posição 
'quote') serão criadas

Caso exista o log operacional, então essa estratégia, com este setup, já foi testada em um momento passado, sendo 
necessário apenas continuar o teste deste ponto até o moento presente; assim, deve-se calcular o 'from_date_time', 
levando em conta a últmima amostra obtida no log, bem como


'''

op_log_table_name = 'btt_' + strategy + '_' + setup + '_log_monitoring'

#Retorna a última linha da tabela de logs, caso exista.
oper_log = Log_handler(op_log_table_name).get(limit=1)

#Cria o dataframe de log

if (len(oper_log) == 0):
    
    #Busca as tabelas de dados inteiras, arrumando-as em um dicionário e deixando todas a partir do menor 'timestamp' 
    #inicial dentre as tabelas buscadas
    mkt_data = get_klines_from_db(setup_parameters['work_pairs'][0])
    
    #Arma a primeira linha do log, com todos os pares na posição quote

else:
    
    #date_time = #calcula a partir de que amostra deve buscar no banco, utilizando a linha de log retornada, 
                 #bem como os parâmetros da estratégia
    
    mkt_data = get_klines_from_db(setup_parameters['work_pairs'][0], from_date_time=date_time)

#Ordem do loop:
     #Externo: Avançar nas linhas (um open_time, de cada par, por vez)
        #Interno: Varrer todos os pares do dicionário


for i in range (len(mkt_data)):
    
    #Testa stop
    
    #Parse dos dados para o candle_interval de trabalho
    
    #Testa indicadores e posiciona
    
    #Verifica se há difenrença entre posição corrente e posição indicada
    
    #


#Procura a tabela de logs, se não existir, as cria

In [42]:
#Teste main

In [81]:
class Log_handler():
    
    def __init__(self, strategy, setup):
        
        self.strategy = strategy
        self.setup = setup
        
        

    def get(self, **kwargs):
        
        limit = kwargs.get('limit')
    
    
    
    
    
    
    def read_table(table_name, **kwargs):
    
    """Busca no banco a tabela [table_name], retornando uma lista, tal que cada elemento representa
    uma linha (registro) da tabela, ou uma lista vazia, caso a tabela não seja encontrada. Pode retornar
    a tabela inteira, ou parte dos valores se forem informados, corretamente, os argumentos opcionais
    **kwargs.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser buscada
    
    **kwargs:
    =========
    
    mute -- Suprime o log se 'yes'
    
    field_key -- Chave (nome da coluna) do campo tomado como referência de ordenamento
    
    sort_type      -- Modo como os registros são buscados:
        
    *sort_type options*
     -----------------
        - DESC ==> Do maior para o menor
        - ASC  ==> Do menor para o maior
        
    limit     -- Número de registros que devem ser retornados
    """
    
    field_key = kwargs.get('field_key')

SyntaxError: invalid syntax (<ipython-input-81-e68f0c2786ab>, line 1)

In [ ]:
#Teste estratégia alpha_1
from modules import indicators

In [ ]:
getattr(indicators, 'f1')()

In [ ]:
pair = 'LTCUSDT'

data = data_dict[pair]['high']

teste = data.rolling(window=20).mean()

print(teste)